In [1]:
import scipy.ndimage.filters
import numpy as np
from skimage import feature
import glob
import matplotlib.pyplot as plt

from PIL import Image

from sklearn.externals import joblib
import tensorflow as tf
# from nets import inception
# from preprocessing import inception_preprocessing


C:\Users\anik\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# image to features
def feature_computation(image_arr):
    
#     image to numpy array

#     image = Image.fromarray(image_arr)

    image = image_arr   
    
    
    features = []
    laplacian0 = scipy.ndimage.filters.laplace(image[:,:,0])
    laplacian1 = scipy.ndimage.filters.laplace(image[:,:,1])
    laplacian2 = scipy.ndimage.filters.laplace(image[:,:,2])
    
#     lap_mat0 = np.matrix(laplacian[:,:,0])
#     lap_mat1 = np.matrix(laplacian[:,:,1])
#     lap_mat2 = np.matrix(laplacian[:,:,2])

    lap_mat0 = laplacian0
    lap_mat1 = laplacian1
    lap_mat2 = laplacian2
    
    
    
    features.append(lap_mat0.sum())
    features.append(lap_mat1.sum())
    features.append(lap_mat2.sum())
    
#     image_gray = image.convert('LA')
    image_gray = (image[:,:,0] + image[:,:,1] + image[:,:,2])/3
#     gray_array = np.array(image_gray)[:,:,0]
    gray_array = image_gray
    edges = feature.canny(gray_array,sigma = 3,low_threshold=5, high_threshold=20)
    max_val = np.amax(edges)

    
    for j in range(0,edges.shape[1]):
        for i in range(0,edges.shape[0]):
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i+1][j] < .1*max_val:
                    break
        for ii in range(0,edges.shape[0]):
            i = edges.shape[0] - ii -1
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i-1][j] < .1*max_val:
                    break
                    
    features.append(edges.sum())
    
    return features

    

    
    

In [21]:
def random_forest_predict_probability(image_arr):
    
#     image = Image.fromarray(image_arr, 'RGB')


#     ind1,ind2,ind3,ind4 = image_arr_list.shape



#     print("print input inside predictor "+str(image_arr_list.shape))
#     print("print input list's length "+str(ind1))
    
    predictions = []
    feature_vec = feature_computation(image_arr)
    model = joblib.load("radom_forest_model.pkl")
    print(model.classes_)
    single_prediction = model.predict_proba([feature_vec])
    predictions.append(single_prediction)

            
    
    return predictions
    

In [22]:
image_for_explanation=[]
for filename in glob.glob('./explain_images/*.png'):
    im=Image.open(filename)
    image_for_explanation.append(im.copy())

#     image_raw = tf.image.decode_png(im, channels=3)
#     image_for_explanation(image_raw.copy())
    im.close()
    
    
image = image_for_explanation[2]
id1 , id2 = image.size

im_arr = np.asarray(image)
# im_arr_arr = np.asarray([im_arr])

# print("image prediction:",str(random_forest_predictor(im_arr_arr)))
# print("print image array size "+str(im_arr.shape))
# im_resized = np.reshape(im_arr,(id1,id2,3))
# print("print array after resized "+str(im_resized.shape))


test_image =  im_arr

print(test_image.shape)

random_forest_predict_probability(test_image)

(106, 100, 3)
[False  True]


[array([[0.64, 0.36]])]

In [16]:
model = joblib.load("radom_forest_model.pkl")
model.classes_


array([False,  True])